Lo siguiente está basado en los libros:

* B. Rumbos, Pensando Antes de Actuar: Fundamentos de Elección Racional, 2009

* G. J. Kerns, Introduction to Probability and Statistics Using R, 2014.

El libro de G. J. Kerns tiene github: [jkerns/IPSUR](https://github.com/gjkerns/IPSUR)


**Nota:**

* En algunas líneas no es necesario colocar `print` y sólo se ha realizado para mostrar los resultados de las funciones en un formato similar al de R pues la nota se escribió con *jupyterlab* y *R*.

In [1]:
options(repr.plot.width=5, repr.plot.height=5) #esta línea sólo se ejecuta para jupyterlab con R

# Introducción

El objetivo de la Estadística es la de descripción de fenómenos que a través de datos manifiestan variabilidad. En particular, la inferencia estadística **describe alguna característica de una población** en base a la **información que se tiene de una o varias muestras de la misma**. 

**Un poco de historia ...**

Ronald Aylmer Fisher (1890 - 1962), matemático inglés, cuyo interés por la agricultura y la genética lo llevó a dirigir el centro de investigación agrícola más importante de su época en el Reino Unido, el *Rothamsted Agricultural Experiment Station*, contribuyó gran parte de las ideas fundamentales que utiliza la estadística moderna y originó lo que se conoce como **diseño de experimentos**. La **aleatoriedad del muestreo** fue introducida por él, formalizó lo que hoy conocemos como **pruebas de hipótesis** y acuñó el término de **hipótesis nula**, entre otras aportaciones.

# Hipótesis y evidencia

De acuerdo al filósofo austriaco Karl Popper (1902 - 1994) el conocimiento se construye, a partir de la imposibilidad de la negación de las hipótesis que se introducen para explicar la realidad. **Una hipótesis no puede desecharse, a menos que haya evidencia suficiente para ponerla en duda**.

Usualmente las hipótesis surgen para tratar de explicar algún fenómeno, por ejemplo:

1) **Fenómeno:** amanecer. Hipótesis: **el sol sale todos los días**.

2) **Fenómeno:** un paciente llega a ver a una médica y le explica sus síntomas. Hipótesis: **paciente enfermo de la enfermedad "X"**.

3) **Fenómeno:** estudiante realiza examen de admisión a un departamento "Y" de una universidad. Hipótesis: **admisión al departamento "Y" es sin sesgo**.

Las hipótesis **son rechazadas o no dependiendo de los datos observados basados en una o varias muestras (evidencia)**. Las hipótesis iniciales son nombradas **hipótesis nulas** (no hay cambio, no hay diferencia, no hay característica especial...). Para cada ejemplo anterior supóngase que cada hipótesis es la inicial  o nula y además se tienen las siguientes evidencias:

1) Una persona observa la salida del sol en la mañana por su ventana. 

2) La médica reconoce síntomas relacionados a la enfermedad "X".

3) La estudiante revisa los resultados del examen de admisión.

Para cada evidencia se calculan probabilidades de la evidencia dada la hipótesis (y puede ser el cálculo por un modelo equiprobable, subjetivo o frecuentista, por ejemplo):

1) Probabilidad que el sol salga el día de mañana dado que el sol sale todos los días: 1.

2) Probabilidad que el paciente tenga tales síntomas dado que el paciente tiene la enfermedad "X": 0.8.

3) Probabilidad de ser admitida en el departamento "Y" dado que la admisión a tal departamento es sin sesgo: 0.4.

Entonces cada hipótesis nula se rechaza (la evidencia **sugiere** fuertemente que es falsa) o no se rechaza (la evidencia **no sugiere** fuertemente que es falsa) en favor o no, respectivamente, de la **hipótesis alterna** (complementaria de la hipótesis nula). Para cada ejemplo anterior se tienen como ejemplo las siguientes hipótesis alternas:

* El sol no sale todos los días.

* El paciente no tiene la enfermedad "X".

* Admisión al departamento "Y" tiene sesgo.

Las **pruebas o contraste de hipótesis** son una metodología estadística que verifican la validez de las hipótesis o infieren características de una población (por ejemplo **parámetros** si las hipótesis están enunciadas en términos de parámetros poblacionales) al ser puestas a prueba con la evidencia observada u obtenida vía muestras de la población (datos).

**Comentarios:** 

* El no rechazo/rechazo de la hipótesis nula **no garantiza** que las hipótesis nulas/alternas sean verdaderas pues que la evidencia (datos) no contradigan las hipótesis no implica su veracidad (principalmente porque la evidencia también tiene incertidumbre). Por ejemplo, no rechazar la hipótesis que el paciente tiene la enfermedad "X" dada la evidencia **no implica** que el paciente efectivamente esté enfermo de la enfermedad "X" (podría tener otra enfermedad relacionada); asimismo, rechazar que el proceso de admisión al departamento "Y" a estudiantes no tiene sesgo **no sugiere** que exista sesgo al proceso de admisión para estudiantes (podría el estudiante no haber cumplido con el perfil requerido).

* La notación para la hipótesis inicial o nula es $H_0$ y la alternativa es $H_a$.

* Es importante definir $H_0$ de modo que su rechazo nos proporcione información útil y relevante. Por ejemplo, si se especifica la hipótesis nula: "el $90\%$ de los crímenes cometidos en la ciudad de México quedan impunes", la hipótesis alterna es: "el porcentaje que queda impune es distinto del $90\%$" y el rechazo de la hipótesis nula en favor de la alterna no es muy útil. En este ejemplo podría ser $H_0$: "Al menos un $90\%$ de los crímenes cometidos en la ciudad de México quedan impunes".

## Nivel de significancia de la prueba

El rechazo o no rechazo de la hipótesis nula conduce a errores si tal hipótesis es verdadera o falsa respectivamente. Neyman y Pearson nombraron a tales errores como errores de tipo 1 y 2:

|$H_0$|Rechazar $H_0$|No rechazar $H_0$|
|:------:|:----:|:----:|
|Verdadera|Error tipo I|-|
|Falsa|-|Error tipo II|

y en el proceso estadístico de pruebas de hipótesis se **considera peor el error tipo I que el II** por ello se controla la probabilidad del error tipo I, el cual se define como la **significancia de la prueba**:

$$\text{nivel de significancia de la prueba es: } P(ETI) = \alpha$$

donde: $ET1$ es el error tipo 1 y típicamente $\alpha$ es $0.05, 0.01$. Tales valores están relacionados con el hecho que en la mayoría de los casos la distribución de los datos es aproximada por una normal y para tal distribución, el $95\%$ de los datos se encuentran a una distancia de $2\sigma$ (aprox.) de la media y el $99\%$ a una distancia de $3\sigma$ (aprox.) y estos valores son particularmente fáciles de calcular sin ninguna ayuda computacional.


**Comentarios:** 

* El $ET1$ puede considerarse como encarcelar a una persona inocente y el $ET2$ como no sentenciar a una persona culpable (el $ET1$ el "peor error"). El error tipo II típicamente se denota como $\beta$.

* $H_0$ típicamente no es rechazada a menos que la evidencia sugiera fuertemente que es falsa (es significativa).

* Una vez fijada la probabilidad del error tipo I el proceso de Neyman y Pearson minimiza la probabilidad del error tipo II: no rechazar cuando $H_0$ es falsa la cual equivale a maximizar la probabilidad de rechazar $H_0$ cuando $H_0$ es falsa: $P(ETII) = P(\{\text{no rechazar } H_0\}|\{H_0 \text{ es falsa}\})=1-P(\{\text{rechazar} H_0\}|\{H_0 \text{ es falsa}\})$. A esta última probabilidad se le conoce como **potencia de la prueba** y está dada por $1-\beta$. Maximizar la potencia es entonces maximizar la probabilidad de rechazo de una hipótesis falsa.

* Evidentemente nos gustaría que la probabilidad de ambos tipos de error fuese lo más baja posible, sin embargo, para **una muestra dada** existe un trueque inevitable entre las dos probabilidades $\alpha$ y $\beta$. Si una disminuye la otra aumenta y viceversa. Una vez más, depende del experimentador el tomar la decisión adecuada. Si dado un tamaño de muestra y un valor de significancia $\alpha$, puede elegirse entre pruebas con distinta potencia, entonces lo óptimo es escoger la que tiene la potencia máxima.

    Lo siguiente es parte del artículo original de Neyman y Pearson: "¿Qué es más serio, condenar a un inocente o exonerar a un culpable? Esto dependerá de las consecuencias del error: ¿se castigará con la muerte o simplemente con una multa? ¿Cuál es el peligro a la comunidad de tener un criminal suelto? Desde el punto de vista de la teoría matemática, lo único que podemos lograr es demostrar como el riesgo de errores puede ser controlado y minimizado."

## Valor p o cálculo de la probabilidad condicional dado que $H_0$ es verdadera

En varios ejemplos para diferentes valores del nivel de significancia de la prueba, $\alpha$, suele darse la situación que se rechaza $H_0$ para un valor de $\alpha$ y para otro valor no se rechaza $H_0$ por lo que nuestra decisión final depende de $\alpha$. ¿Cuál es el valor entonces que se debe elegir $.05$ o $.01$? ¿cuál es el nivel de significancia para el cual se rechazaría a cualquier nivel de significancia más alto y cuál es el nivel para el que cualquier valor de nivel de significancia más bajo no se rechazaría?.

El **valor p** o **nivel de significancia observado** de una prueba de hipótesis, es la probabilidad de **obtener los datos o evidencia, a partir de una muestra**, dado que la hipótesis nula es verdadera.

### Ejemplo

Imaginemos que un estudiante realiza un examen de matemáticas de opción múltiple. El examen consta de $10$ preguntas, cada una con $5$ opciones de respuesta, de las cuales sólo una es la correcta. Este estudiante obtiene $5$ preguntas correctas. 

Notemos que si las respuestas se eligen en forma aleatoria, la probabilidad de éxito es de $1$ para cada pregunta. Podemos pensar al proceso de contestar el examen aleatoriamente como un experimento binomial con $10$ ensayos y probabilidad de éxito de $\frac{1}{5}$ en cada ensayo y $X$ variable aleatoria binomial $n=10, p=\frac{1}{5}$. Por tanto el número de aciertos esperados al contestar de forma aleatoria es de:

$$\mu = (10) \frac{1}{5}.$$

¿Qué tan plausible es que el estudiante haya obtenido al menos $5$ aciertos? ¿Será meramente producto del azar o quizás posee algún conocimiento de matemáticas? Para contestar estas preguntas planteamos las siguientes hipótesis complementarias:

$H_0$ : el estudiante contestó al azar

$H_a$ : el estudiante no contestó al azar.

La probabilidad de tener al menos 5 aciertos puede calcularse como, $$P(X \geq 5) = 1 - P(X < 5)=1-\displaystyle \sum_{i=0}^4P(X=i)$$

Si $H_0$ es cierta y $p=\frac{1}{5}$ entonces:

In [69]:
pbinom(5,10,1/5,lower.tail = F) + dbinom(5,10,1/5)

[1] 0.0327935

**Obs:** el valor anterior es el **valor p**.

En conclusión, la probabilidad de que el estudiante haya obtenido al menos $5$ aciertos contestando de manera aletoria es de $3.3\%$. Existen dos posibilidades:

* $H_0$ es verdadera y estamos ante un acontecimiento inusual que sólo ocurre el $3.3\%$ del tiempo o bien,

* $H_0$ es falsa.

Si $\alpha=.01$ no rechazamos y si $\alpha=.05$ sí rechazamos de acuerdo al valor p. En el segundo caso, tenemos que $H_0$ se rechazaría en favor de $H_a$. Si consideramos que los $5$ aciertos no fueron obra de la casualidad, entonces, ¿podemos concluir que el estudiante sabe algo de matemáticas? Desafortunadamente no, ya que no puede descartarse que haya copiado algunas de las respuestas. Intuitivamente, los $5$ aciertos obtenidos podrían ser suficientemente significativos para descartar que son producto del azar.

**Comentarios:**

* El valor p nos proporciona el límite entre rechazar y no rechazar de acuerdo a la regla: si el valor p es menor que el nivel de significancia de la prueba $\alpha$ entonces rechazamos $H_0$. En este sentido, el valor p es el valor mínimo para el cual el valor de la muestra observada conduce a rechazar $H_0$ ($\alpha$'s más grandes que el valor p conducen a rechazar $H_0$).

* Un valor p pequeño, no favorece a la hipótesis nula, un valor p grande indica que los datos obtenidos son compatibles con la hipótesis nula.

## Región de rechazo y estadísticas de prueba

La región de rechazo o región crítica $\mathscr{C}$ es el subconjunto de valores del espacio muestral (aquel que contiene diferentes muestras de la población) para los cuales se rechaza $H_0$. El procedimiento estadístico de pruebas de hipótesis entonces fija la probabilidad del error tipo 1 como: $$\alpha = P(ET1) = P(\{x_1,\dots,x_n\} \in \mathscr{C} | \{H_0 \text{ es verdadera}\}).$$

y se rechaza si $\alpha$ es pequeño. En las pruebas de hipótesis **paramétricas** (población completamente definida en términos de parámetros, por ejemplo $\mu$ o $\sigma$) la probabilidad (condicional) anterior típicamente se reformula en términos de una estadística resúmen, por ejemplo $\bar{X}$: $$\alpha = P(ET1) = P(\bar{X} \in \mathscr{C} | \{H_0 \text{ es verdadera}\}).$$

y a esta estadística se le nombra **estadística de prueba**.

### Ejemplo

Fisher originalmente construyó las pruebas de hipótesis para probar la eficacia de fertilizantes y pesticidas en experimentos agrícolas para evaluar la productividad del campo en cuestión. Supongamos que quiere determinarse si la adopción del uso de cierto fertilizante para el maíz es deseable. Los pasos a seguir son los siguientes:

* Diseñar un experimento con una muestra de parcelas en las que se aplica el fertilizante.

* Elegir una cantidad que represente a los datos que nos interesa medir, usualmente se trata de una media, de una desviación estándar o de una proporción. En este caso queremos representar el rendimiento o productividad de las parcelas. La cantidad a medir supóngase que es la cantidad promedio de maíz producida, $\mu$, el rendimiento promedio. Sea $R_0$ el valor actual del rendimiento promedio del maíz. 

* Determinar la hipótesis nula y la alterna, que en este caso son: $H_0:$ el fertilizante no sirve, $H_a:$ el fertilizante sirve. En términos del rendimiento promedio éstas se reescriben como:

$$H_0: \mu = R_0$$

$$H_a: \mu > R_0$$

* Establecer un nivel de significancia, por ejemplo $\alpha = 0.05.$

* Obtener los datos acerca de los rendimientos de cada parcela de la muestra y calcular el rendimiento promedio $\bar{x}$. Tomando $\bar{X}$ como estadística de prueba si $\bar{x} \leq R_0$ entonces $H_0$ no será rechazada y por tanto se desea calcular: $P(\bar{X} > \bar{x}|\{H_0 \text{ es verdadera })$ el cual es el valor p.

* Si el valor p es menor a $0.05$, entonces concluimos que estamos ante algo muy inusual, que sólo ocurre con probabilidad menor al $5\%$ o bien, la hipótesis nula es falsa. Se dice que los resultados son significativos a un nivel de $0.05$. Dado que $0.05$ es el nivel de significancia preestablecido, se procede a rechazar la hipótesis nula -en favor de la alterna-, es decir, el fertilizante sirve.


## Ejemplo en R...

Supóngase que la proporción de estudiantes que son admitidos a la Universidad Berkley de California está denotado por *prop* y la oficina de relaciones públicas emite un comunicado en el que se menciona que la UCB ha tenido un $40\%$ de aceptación. Considérese los datos *UCBAdmissions* de *R* que contienen la admisión del año $1973$ por género, departamento y la variable *admitido* y *no admitido*:

In [72]:
print(dimnames(UCBAdmissions))

$Admit
[1] "Admitted" "Rejected"

$Gender
[1] "Male"   "Female"

$Dept
[1] "A" "B" "C" "D" "E" "F"



Supóngase que tales datos constituyen una muestra aleatoria simple. Responder: ¿tales datos son consistentes con la carta emitida por relaciones públicas o proveen evidencia que la tasa de aceptación es significativamente menor al $40\%$? (usar un nivel de significancia de $\alpha=0.01$.

**Solución:**

La hipótesis nula es $H_0: prop=0.4$ y la hipótesis alternativa es $H_a: prop<0.4$. Rechazamos la hipótesis nula si el valor p es menor a $\alpha = 0.01$. El valor p se calcula como: $$P(\hat{p} < \hat{p}_{obs}|H_0 \text{ es verdadera })$$

donde: $\hat{p}$ es la estadística de prueba (proporción estimada) y $\hat{p}_{obs}$ es la proporción calculada/observada en los datos.

Suponiendo normalidad (garantizada por el teorema del límite central) y que $H_0$ es verdadera resulta: $\frac{\hat{p} - 0.4}{\sqrt{\frac{.4(1-.4)}{n}}} \sim N(0,1)$ y $\hat{p}_{obs}$ se calcula con los datos:

In [101]:
admitted_rejected <- margin.table(UCBAdmissions, 1) #el número 1 indica que se 
                                               #agregue por género y departamento
                                               #si en lugar del número 1 se usa
                                               #c(1,2) sólo se agrega por departamento
                                               #es equivalente a: apply(UCBAdmissions, 1, sum) 
print(admitted_rejected)
print('proporcion de admitidos en los datos de UCBAdmissions:')
p_obs<-as.numeric(admitted_rejected[1]/(admitted_rejected[1]+admitted_rejected[2]))
print(p_obs)

Admit
Admitted Rejected 
    1755     2771 
[1] "proporcion de admitidos en los datos de UCBAdmissions:"
[1] 0.3877596


por lo que $\hat{p}_{obs}=\frac{1755}{1755+2771} = 0.3878$ y se tiene:

In [105]:
n<-as.numeric(admitted_rejected[1]+admitted_rejected[2]) #número de estudiantes
p_H_0<-0.4 #proporción bajo hipótesis H0 es verdadera
cuantil<-(p_obs-p_H_0)/sqrt((p_H_0)*(1-p_H_0)/n)
print(cuantil)

[1] -1.680919


y el valor p entonces es: $$P(\hat{p} < \hat{p}_{obs}) = P\left(\frac{\hat{p} - 0.4}{\sqrt{\frac{.4(1-.4)}{n}}} < -1.681\right) = P(Z < -1.68),$$ donde $Z \sim N(0,1)$.

In [106]:
pnorm(cuantil)

[1] 0.04638929

Y para el nivel de significancia especificado $\alpha=0.1$ **no se rechaza** $H_0$ en favor de la hipótesis: "la proporción verdadera de estudiantes admitidos a la Universidad es menor al $40\%$" y los datos observados son consistentes con el comunicado.

**Obs:** si $\alpha=0.05$ entonces sí hubiéramos rechazado $H_0$ y hubiéramos concluido que los datos UCBAdmissions dan evidencia significativa que la proporción verdadera de estudiantes que son admitidos a la Universidad es menor al $40\%$. Depende de cada situación el nivel de significancia fijado, ¿se trata de una situación de vida o muerte o de algo con consecuencias menos drásticas?